In [2]:
import torch
import torch.nn as nn

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [3]:
import sys, os
cur_dir = os.getcwd()
test_dir = os.path.join(cur_dir, os.pardir)
src_dir = os.path.join(test_dir, os.pardir)
sys.path.append(os.path.abspath(src_dir))

In [4]:
from data.processed.CelebA_1024.call_celeba import CelebALoader
cl = CelebALoader()

In [5]:
df = cl.load_anno('list_attr_celeba_1024.txt')

In [6]:
df

,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,Blond_Hair,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
000001.jpg,-1,1,1,-1,-1,-1,-1,-1,-1,-1,...,-1,1,1,-1,1,-1,1,-1,-1,1
000002.jpg,-1,-1,-1,1,-1,-1,-1,1,-1,-1,...,-1,1,-1,-1,-1,-1,-1,-1,-1,1
000003.jpg,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,...,-1,-1,-1,1,-1,-1,-1,-1,-1,1
000004.jpg,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,1,-1,1,-1,1,1,-1,1
000005.jpg,-1,1,1,-1,-1,-1,1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,1,-1,-1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
001020.jpg,1,1,1,1,-1,-1,-1,1,-1,-1,...,-1,1,-1,1,-1,-1,-1,-1,1,1
001021.jpg,-1,-1,1,-1,-1,-1,1,-1,-1,-1,...,-1,-1,-1,1,1,-1,1,-1,-1,1
001022.jpg,1,-1,-1,-1,-1,-1,-1,1,-1,-1,...,1,-1,-1,-1,-1,-1,-1,-1,-1,1
001023.jpg,-1,1,1,-1,-1,-1,1,-1,-1,-1,...,-1,1,-1,1,-1,-1,1,-1,-1,1


In [69]:
y = df['Male'].replace(to_replace=-1, value=0)
y = torch.tensor(y.values)
y = nn.functional.one_hot(y, num_classes=2).type(torch.FloatTensor)

In [8]:
imgs = cl.load_img()

In [9]:
imgs[0].shape

torch.Size([218, 178])

In [10]:
for i in range(1024):
    imgs[i].unsqueeze_(dim=0).unsqueeze_(dim=0)
imgs[0].shape

torch.Size([1, 1, 218, 178])

In [11]:
conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
pool = nn.MaxPool2d(2)

In [12]:
input = torch.Tensor(1, 1, 218, 178)

In [15]:
out1 = conv1(input)
out2 = conv2(out1)
out3 = pool(out2)

print(out1.shape)
print(out2.shape)
print(out3.shape)

torch.Size([1, 32, 218, 178])
torch.Size([1, 64, 218, 178])
torch.Size([1, 64, 109, 89])


In [16]:
out4 = out3.view(out3.size(0), -1)
print(out4.shape)

torch.Size([1, 620864])


In [18]:
fc = nn.Sequential(nn.Linear(620864, 300), nn.GELU(), nn.Linear(300, 2))
out5 = fc(out4)
print(out5.shape)

torch.Size([1, 2])


In [55]:
layer1 = nn.Sequential(nn.Conv2d(1, 32, kernel_size=3, padding=1),
                                    nn.GELU(),
                                    nn.MaxPool2d(kernel_size=2, stride=2))
layer2 = nn.Sequential(nn.Conv2d(32, 64, kernel_size=3, padding=1),
                                    nn.MaxPool2d(kernel_size=2, stride=2))

In [57]:
outl1 = layer1(input)
outl2 = layer2(outl1)
print(outl1.shape)
print(outl2.shape)

torch.Size([1, 32, 109, 89])
torch.Size([1, 64, 54, 44])


In [58]:
outl3 = outl2.view(out3.size(0), -1)
print(outl3.shape)

torch.Size([1, 152064])


In [59]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
                                    nn.GELU(),
                                    nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
                                    nn.GELU(),
                                    nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(152064, 2)

        nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [73]:
model = CNN()

In [74]:
loss = nn.CrossEntropyLoss()
opt = torch.optim.Adam(model.parameters())

In [75]:
epochs = 3
for epoch in range(epochs):
    for i in range(1024):
        opt.zero_grad()
        hypothesis = model(imgs[i])
        cost = loss(hypothesis, y[i].unsqueeze(dim=0))
        cost.backward()
        opt.step()
        if i == 0 and epoch == 0:
            print("Primary Loss: {:.6f}".format(epoch, cost.item()))
        if i % 512 == 256:
            print("Epoch: {} | Loss: {:.6f}".format(epoch, cost.item()))
        

    print("Epoch: {} | Loss: {:.6f}".format(epoch, cost.item()))

Epoch: 0 | Primary Loss: 0.827558
Epoch: 0 | Loss: 0.312660
Epoch: 0 | Loss: 0.011848
Epoch: 0 | Loss: 0.000600
Epoch: 1 | Loss: 0.002530
Epoch: 1 | Loss: 0.011676
Epoch: 1 | Loss: 0.000489
Epoch: 2 | Loss: 0.064049
Epoch: 2 | Loss: 0.000001
Epoch: 2 | Loss: 0.000000


In [76]:
acc = 0
for i in range(1024):
    hypothesis = model(imgs[i])
    if torch.argmax(hypothesis) == torch.argmax(y[i].unsqueeze(dim=0)):
        acc += 1
print(acc / 1024)

0.9501953125


In [ ]:
batch_iter = 8
batch_size = int(1024 / batch_iter)
imgs_ts = torch.FloatTensor(imgs)
imgs_batch = imgs_ts.split(batch_size, dim=0)
y_batch = y.split(batch_size, dim=0)

In [ ]:
epochs = 10
for epoch in range(epochs):
    for batch in range(batch_iter):
        opt.zero_grad()
        hypothesis = model(imgs_batch)
        cost = loss(hypothesis, y_batch)
        cost.backward()
        opt.step()